In [1]:
import cv2
import mediapipe as mp
import numpy as np
from collections import defaultdict

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

# Load your face image (change path here)
image = cv2.imread("samples/brad_front.png")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
h, w, _ = image.shape

# Process the image to detect face landmarks
results = face_mesh.process(image_rgb)
if not results.multi_face_landmarks:
    raise ValueError("No face detected")

landmarks = results.multi_face_landmarks[0].landmark

# Convert landmarks to numpy array (x,y,z)
points = np.array([(lm.x * w, lm.y * h, lm.z * w) for lm in landmarks])

# Flip coords for Blender axis system and recenter at origin
verts_fixed = np.array([(x, h - y, -z) for (x, y, z) in points])
centroid = verts_fixed.mean(axis=0)
verts_centered = verts_fixed - centroid

# UV coordinates (u,v) with v flipped for correct texture orientation
uvs = np.array([(lm.x, 1 - lm.y) for lm in landmarks])

# Build adjacency dict from FACEMESH_TESSELATION edges
edges = mp_face_mesh.FACEMESH_TESSELATION

adj = defaultdict(set)
for start, end in edges:
    adj[start].add(end)
    adj[end].add(start)

# Find triangles by intersecting adjacency sets
triangles = set()
for a in adj:
    for b in adj[a]:
        common = adj[a].intersection(adj[b])
        for c in common:
            tri = tuple(sorted((a, b, c)))
            triangles.add(tri)
triangles = list(triangles)


# Write OBJ file with texture references
def write_obj(filename, verts, uvs, faces, texture_filename):
    with open(filename, "w") as f:
        f.write(f"mtllib {texture_filename}.mtl\n")
        f.write("usemtl material_0\n")
        for v in verts:
            f.write(f"v {v[0]:.6f} {v[1]:.6f} {v[2]:.6f}\n")
        for uv in uvs:
            f.write(f"vt {uv[0]:.6f} {uv[1]:.6f}\n")
        for tri in faces:
            f.write(
                f"f {tri[0]+1}/{tri[0]+1} {tri[1]+1}/{tri[1]+1} {tri[2]+1}/{tri[2]+1}\n"
            )


def write_mtl(filename, texture_filename):
    with open(filename, "w") as f:
        f.write("newmtl material_0\n")
        f.write("Ka 1.000 1.000 1.000\n")
        f.write("Kd 1.000 1.000 1.000\n")
        f.write("Ks 0.000 0.000 0.000\n")
        f.write(f"map_Kd {texture_filename}\n")


texture_image_name = "face_texture.jpg"
cv2.imwrite(texture_image_name, image)

obj_filename = "face_mesh.obj"
mtl_filename = "face_mesh.mtl"

write_obj(obj_filename, verts_centered, uvs, triangles, mtl_filename)
write_mtl(mtl_filename, texture_image_name)

print(f"OBJ saved: {obj_filename}")
print(f"MTL saved: {mtl_filename}")
print(f"Texture saved: {texture_image_name}")

OBJ saved: face_mesh.obj
MTL saved: face_mesh.mtl
Texture saved: face_texture.jpg


I0000 00:00:1745237858.448858   28484 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1745237858.452370   28641 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745237858.458726   28641 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745237858.513595   28642 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


In [3]:
import cv2
import mediapipe as mp
import numpy as np
from collections import defaultdict

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

image = cv2.imread("samples/brad_front.png")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
h, w, _ = image.shape

results = face_mesh.process(image_rgb)
if not results.multi_face_landmarks:
    raise ValueError("No face detected")

landmarks = results.multi_face_landmarks[0].landmark

points = np.array([(lm.x * w, lm.y * h, lm.z * w) for lm in landmarks])
verts_fixed = np.array([(x, h - y, -z) for (x, y, z) in points])
centroid = verts_fixed.mean(axis=0)
verts_centered = verts_fixed - centroid

scale = 0.002  # keep original scale or adjust as needed
verts_scaled = verts_centered * scale

uvs = np.array([(lm.x, 1 - lm.y) for lm in landmarks])

# Collect edges from Tessellation, Contours, and Face Oval
edges = set(mp_face_mesh.FACEMESH_TESSELATION)
edges.update(mp_face_mesh.FACEMESH_CONTOURS)
edges.update(mp_face_mesh.FACEMESH_FACE_OVAL)

adj = defaultdict(set)
for start, end in edges:
    adj[start].add(end)
    adj[end].add(start)

triangles = set()
for a in adj:
    for b in adj[a]:
        common = adj[a].intersection(adj[b])
        for c in common:
            tri = tuple(sorted((a, b, c)))
            triangles.add(tri)

# Define iris boundary landmark indices from MediaPipe docs
left_eye = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]
right_eye = [
    362,
    382,
    381,
    380,
    374,
    373,
    390,
    249,
    263,
    466,
    388,
    387,
    386,
    385,
    384,
    398,
]


# Simple fan triangulation to close eye holes
def add_eye_cap(triangles, eye_indices):
    center = eye_indices[0]
    for i in range(1, len(eye_indices) - 1):
        tri = tuple(sorted((center, eye_indices[i], eye_indices[i + 1])))
        triangles.add(tri)


add_eye_cap(triangles, left_eye)
add_eye_cap(triangles, right_eye)


# add_eye_caps(triangles, left_iris)
# add_eye_caps(triangles, right_iris)

triangles = list(triangles)


def write_obj(filename, verts, uvs, faces, texture_filename):
    with open(filename, "w") as f:
        f.write(f"mtllib {texture_filename}.mtl\n")
        f.write("usemtl material_0\n")
        for v in verts:
            f.write(f"v {v[0]:.6f} {v[1]:.6f} {v[2]:.6f}\n")
        for uv in uvs:
            f.write(f"vt {uv[0]:.6f} {uv[1]:.6f}\n")
        for tri in faces:
            f.write(
                f"f {tri[0]+1}/{tri[0]+1} {tri[1]+1}/{tri[1]+1} {tri[2]+1}/{tri[2]+1}\n"
            )


def write_mtl(filename, texture_filename):
    with open(filename, "w") as f:
        f.write("newmtl material_0\n")
        f.write("Ka 1.000 1.000 1.000\n")
        f.write("Kd 1.000 1.000 1.000\n")
        f.write("Ks 0.000 0.000 0.000\n")
        f.write(f"map_Kd {texture_filename}\n")


texture_image_name = "face_texture.jpg"
cv2.imwrite(texture_image_name, image)

obj_filename = "face_mesh.obj"
mtl_filename = "face_mesh.mtl"

write_obj(obj_filename, verts_scaled, uvs, triangles, mtl_filename)
write_mtl(mtl_filename, texture_image_name)

print(f"OBJ saved: {obj_filename}")
print(f"MTL saved: {mtl_filename}")
print(f"Texture saved: {texture_image_name}")

OBJ saved: face_mesh.obj
MTL saved: face_mesh.mtl
Texture saved: face_texture.jpg


I0000 00:00:1745237910.399195   28484 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
W0000 00:00:1745237910.402604   30312 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745237910.409355   30312 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [4]:
import cv2
import mediapipe as mp
import numpy as np
from collections import defaultdict

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

image = cv2.imread("samples/marilyn_front.png")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
h, w, _ = image.shape

results = face_mesh.process(image_rgb)
if not results.multi_face_landmarks:
    raise ValueError("No face detected")

landmarks = results.multi_face_landmarks[0].landmark

points = np.array([(lm.x * w, lm.y * h, lm.z * w) for lm in landmarks])
verts_fixed = np.array([(x, h - y, -z) for (x, y, z) in points])
centroid = verts_fixed.mean(axis=0)
verts_centered = verts_fixed - centroid

scale = 0.002  # keep original scale or adjust as needed
verts_scaled = verts_centered * scale

uvs = np.array([(lm.x, 1 - lm.y) for lm in landmarks])

# Collect edges from Tessellation, Contours, and Face Oval
edges = set(mp_face_mesh.FACEMESH_TESSELATION)
edges.update(mp_face_mesh.FACEMESH_CONTOURS)
edges.update(mp_face_mesh.FACEMESH_FACE_OVAL)

adj = defaultdict(set)
for start, end in edges:
    adj[start].add(end)
    adj[end].add(start)

triangles = set()
for a in adj:
    for b in adj[a]:
        common = adj[a].intersection(adj[b])
        for c in common:
            tri = tuple(sorted((a, b, c)))
            triangles.add(tri)

# Define iris boundary landmark indices from MediaPipe docs
left_eye = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]
right_eye = [
    362,
    382,
    381,
    380,
    374,
    373,
    390,
    249,
    263,
    466,
    388,
    387,
    386,
    385,
    384,
    398,
]


# Simple fan triangulation to close eye holes
def add_eye_cap(triangles, eye_indices):
    center = eye_indices[0]
    for i in range(1, len(eye_indices) - 1):
        tri = tuple(sorted((center, eye_indices[i], eye_indices[i + 1])))
        triangles.add(tri)


add_eye_cap(triangles, left_eye)
add_eye_cap(triangles, right_eye)

upper_inner_lip = [78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308]
lower_inner_lip = [78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308]
upper_inner_lip.append(upper_inner_lip[0])
lower_inner_lip.append(lower_inner_lip[0])


def add_lip_bridge(triangles, top_idxs, bottom_idxs):
    min_len = min(len(top_idxs), len(bottom_idxs))
    top_idxs = top_idxs[:min_len]
    bottom_idxs = bottom_idxs[:min_len]
    for i in range(min_len - 1):
        t1 = top_idxs[i]
        t2 = top_idxs[i + 1]
        b1 = bottom_idxs[i]
        b2 = bottom_idxs[i + 1]
        triangles.add(tuple(sorted((t1, b1, b2))))
        triangles.add(tuple(sorted((t1, b2, t2))))


add_lip_bridge(triangles, upper_inner_lip, lower_inner_lip)

triangles = list(triangles)


def write_obj(filename, verts, uvs, faces, texture_filename):
    with open(filename, "w") as f:
        f.write(f"mtllib {texture_filename}.mtl\n")
        f.write("usemtl material_0\n")
        for v in verts:
            f.write(f"v {v[0]:.6f} {v[1]:.6f} {v[2]:.6f}\n")
        for uv in uvs:
            f.write(f"vt {uv[0]:.6f} {uv[1]:.6f}\n")
        for tri in faces:
            f.write(
                f"f {tri[0]+1}/{tri[0]+1} {tri[1]+1}/{tri[1]+1} {tri[2]+1}/{tri[2]+1}\n"
            )


def write_mtl(filename, texture_filename):
    with open(filename, "w") as f:
        f.write("newmtl material_0\n")
        f.write("Ka 1.000 1.000 1.000\n")
        f.write("Kd 1.000 1.000 1.000\n")
        f.write("Ks 0.000 0.000 0.000\n")
        f.write(f"map_Kd {texture_filename}\n")


texture_image_name = "face_texture.jpg"
cv2.imwrite(texture_image_name, image)

obj_filename = "face_mesh.obj"
mtl_filename = "face_mesh.mtl"

write_obj(obj_filename, verts_scaled, uvs, triangles, mtl_filename)
write_mtl(mtl_filename, texture_image_name)

print(f"OBJ saved: {obj_filename}")
print(f"MTL saved: {mtl_filename}")
print(f"Texture saved: {texture_image_name}")

OBJ saved: face_mesh.obj
MTL saved: face_mesh.mtl
Texture saved: face_texture.jpg


I0000 00:00:1745237912.595238   28484 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
W0000 00:00:1745237912.598211   30390 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745237912.604512   30392 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
